<a href="https://colab.research.google.com/github/michaellopes16/DataScience/blob/main/Inceptiomtime_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%pip install scikit-learn --upgrade --user

In [ ]:
a = 10
print(f"Valor da variável {a}")

Valor da variável 10


In [ ]:

from builtins import print
import numpy as np
import pandas as pd
import matplotlib
import random

matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'

import os
import operator
#import utils

#from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES
#from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

import keras
import tensorflow as tf 
#from utils.utils import calculate_metrics
#from utils.utils import create_directory
#from utils.utils import check_if_file_exits
import gc
import time

#from utils.utils import save_logs
#from utils.utils import save_test_duration

#from utils.utils import read_all_datasets
#from utils.utils import transform_labels
#from utils.utils import create_directory
#from utils.utils import run_length_xps
#from utils.utils import generate_results_csv

import sys
import sklearn

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/")
!ls

AllCandidas_TEST.csv  AllCandidas_TRAIN.csv  AllCandidas_VAL.csv


In [ ]:
open("/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas_TRAIN.csv")

<_io.TextIOWrapper name='/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas_TRAIN.csv' mode='r' encoding='UTF-8'>

In [ ]:
def check_if_file_exits(file_name):
    return os.path.exists(file_name)


def readucr(filename, delimiter=','):
    print(filename)
    data = pd.read_csv(filename, delimiter=delimiter,header=None)
    Y = data.iloc[:, 652].values
    X = data.iloc[:,0: 652].copy().values
    return X, Y


def readsits(filename, delimiter=','):
    data = np.genfromtxt(filename, delimiter=delimiter)
    Y = data[:, -1]
    X = data[:, :-1]
    return X, Y

In [ ]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            # in case another machine created the path meanwhile !:(
            return None
        return directory_path

In [ ]:
def read_dataset(root_dir, archive_name, dataset_name):
    datasets_dict = {}

    file_name = root_dir + '/archives/' + archive_name + '/' + dataset_name + '/' + dataset_name
    x_train, y_train = readucr(file_name + '_TRAIN.csv')
    #Trocar aqui para teste depois 
    x_test, y_test = readucr(file_name + '_TEST.csv')
    x_val, y_val = readucr(file_name + '_VAL.csv')
    print(f"x_train value:{x_train}")
    print(f"y_train value:{y_train}")
    print(f"x_test value:{x_test}")
    print(f"y_test value:{y_test}")
    print(f"x_val value:{x_val}")
    print(f"y_val value:{y_val}")

    datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                   y_test.copy(), x_val.copy(),y_val.copy())

    return datasets_dict

In [ ]:
def read_all_datasets(root_dir, archive_name):
    datasets_dict = {}

    dataset_names_to_sort = []

    if archive_name == 'TSC':
        for dataset_name in UNIVARIATE_DATASET_NAMES:
            root_dir_dataset = root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
            file_name = root_dir_dataset + dataset_name
            print(file_name)
            x_train, y_train = readucr(file_name + '_TRAIN.csv')
            #Trocar aqui para teste depois
            x_test, y_test = readucr(file_name + '_TEST.csv')
            x_val, y_val = readucr(file_name + '_VAL.csv')
            print(f"Valor do Xtrain no read_all_datasets {x_train}")
            print(f"Valor do y_train no read_all_datasets {y_train}")
            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                           y_test.copy(),x_val.copy(),y_val.copy())

            dataset_names_to_sort.append((dataset_name, len(x_train)))

        dataset_names_to_sort.sort(key=operator.itemgetter(1))

        for i in range(len(UNIVARIATE_DATASET_NAMES)):
            UNIVARIATE_DATASET_NAMES[i] = dataset_names_to_sort[i][0]
    return datasets_dict

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix 
def specificity_score(y_true, y_pred):
    cm  = multilabel_confusion_matrix(y_true, y_pred)
    specificity = []
    for i in range(len(cm)):
        tn = cm[i][0][0]
        fp = cm[i][0][1]
        spec = tn / (tn + fp)
        specificity.append(spec)
    avg_specificity = sum(specificity) / len(specificity)
    return avg_specificity

In [ ]:
precision = []
accuracy = []
recall = []
f1 = []
std = 0

def calculate_metrics(y_test, y_pred, duration):
    res = pd.DataFrame(data=np.zeros((1, 7), dtype=np.double), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','duration','std','specificity'])
    temp = precision_score(y_test, y_pred, average='macro')
    print(f"Temp: {temp}")
    res['precision'] = temp
    res['accuracy'] = accuracy_score(y_test, y_pred)
    res['recall'] = recall_score(y_test, y_pred, average='macro')
    res['f1_score'] = f1_score(y_test, y_pred, average='macro')
    res['specificity'] = specificity_score(y_test, y_pred)
    res['duration'] = duration
    print(res)
    return res

In [ ]:
def save_test_duration(file_name, test_duration):
    res = pd.DataFrame(data=np.zeros((1, 1), dtype=np.double), index=[0],
                       columns=['test_duration'])
    res['test_duration'] = test_duration
    res.to_csv(file_name, index=False)

In [ ]:
def transform_labels(y_train, y_test, y_val):
    """
    Transform label to min equal zero and continuous
    For example if we have [1,3,4] --->  [0,1,2]
    """
    # no validation split
    # init the encoder
    encoder = LabelEncoder()
    # concat train and test to fit
    y_train_test_val = np.concatenate((y_train, y_test, y_val), axis=0)
    # fit the encoder
    encoder.fit(y_train_test_val)
    # transform to min zero and continuous labels
    new_y_train_test = encoder.transform(y_train_test_val)
    # resplit the train and test
    new_y_train = new_y_train_test[0:len(y_train)]
    new_y_test = new_y_train_test[len(y_train):len(y_train)+len(y_test)]
    new_y_val = new_y_train_test[len(y_train)+len(y_test):]
    return new_y_train, new_y_test, new_y_val

In [ ]:
def generate_results_csv(output_file_name, root_dir, clfs):
    res = pd.DataFrame(data=np.zeros((0, 10), dtype=np.float), index=[],
                       columns=['classifier_name', 'archive_name', 'dataset_name', 'iteration',
                                'precision', 'accuracy', 'recall','f1_score' ,'duration','std'])
    for archive_name in UNIVARIATE_ARCHIVE_NAMES:
        datasets_dict = read_all_datasets(root_dir, archive_name)
        for classifier_name in clfs:
            durr = 0.0

            curr_archive_name = archive_name
            for dataset_name in datasets_dict.keys():
                output_dir = root_dir + '/results/' + classifier_name + '/' \
                             + curr_archive_name + '/' + dataset_name + '/' + 'df_metrics.csv'
                #print(output_dir)
                if not os.path.exists(output_dir):
                    continue
                df_metrics = pd.read_csv(output_dir)
                df_metrics['classifier_name'] = classifier_name
                df_metrics['archive_name'] = archive_name
                df_metrics['dataset_name'] = dataset_name
                df_metrics['iteration'] = 0
                res = pd.concat((res, df_metrics), axis=0, sort=False)
                durr += df_metrics['duration'][0]

    res.to_csv(root_dir + output_file_name, index=False)

    res = res.loc[res['classifier_name'].isin(clfs)]

    return res

In [ ]:
def plot_epochs_metric(hist, file_name, metric='loss'):
    plt.figure()
    plt.plot(hist.history[metric])
    plt.plot(hist.history['val_' + metric])
    plt.title('model ' + metric)
    plt.ylabel(metric, fontsize='large')
    plt.xlabel('epoch', fontsize='large')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()

In [ ]:
def save_logs(output_directory, hist, y_pred, y_test, duration,
              lr=True, plot_test_acc=True):
    hist_df = pd.DataFrame(hist.history)
    #2 * (Precision * Recall) / (Precision + Recall)
    print("Histórico")
    print(hist_df)
    hist_df['f1'] = 2 * ((hist_df.iloc[:, 3] * hist_df.iloc[:, 2])/(hist_df.iloc[:, 3] + hist_df.iloc[:, 2]))
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_test, y_pred, duration)
    precision.append(df_metrics['precision'])
    accuracy.append(df_metrics['accuracy'])
    recall.append(df_metrics['recall'])
    f1.append(df_metrics['f1_score'])

    df_metrics.to_csv(output_directory + 'df_metrics_val.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    #if plot_test_acc:
    #    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    #df_best_model['best_model_train_acc'] = row_best_model['acc']
    if plot_test_acc:
        df_best_model['best_model_val_acc'] = row_best_model['val_acc']
    if lr == True:
        df_best_model['best_model_learning_rate'] = row_best_model['lr']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    if plot_test_acc:
        # plot losses
        plot_epochs_metric(hist, output_directory + 'epochs_loss.png')

    return df_metrics

In [ ]:
def create_synthetic_dataset(pattern_len=[0.25], pattern_pos=[0.1, 0.65], ts_len=128, ts_n=128):
    random.seed(1234)
    np.random.seed(1234)

    nb_classes = len(pattern_pos) * len(pattern_len)

    out_dir = '/b/home/uha/hfawaz-datas/dl-tsc/archives/UCRArchive_2018/BinaryData/'

    create_directory(out_dir)

    x_train = np.random.normal(0.0, 0.1, size=(ts_n, ts_len))
    x_test = np.random.normal(0.0, 0.1, size=(ts_n, ts_len))

    y_train = np.random.randint(low=0, high=nb_classes, size=(ts_n,))
    y_test = np.random.randint(low=0, high=nb_classes, size=(ts_n,))

    # make sure at least each class has one example
    y_train[:nb_classes] = np.arange(start=0, stop=nb_classes, dtype=np.int32)
    y_test[:nb_classes] = np.arange(start=0, stop=nb_classes, dtype=np.int32)

    # each class is defined with a certain combination of pattern_pos and pattern_len
    # with one pattern_len and two pattern_pos we can create only two classes
    # example:  class 0 _____-_  & class 1 _-_____

    # create the class definitions
    class_def = [None for i in range(nb_classes)]

    idx_class = 0
    for pl in pattern_len:
        for pp in pattern_pos:
            class_def[idx_class] = {'pattern_len': int(pl * ts_len),
                                    'pattern_pos': int(pp * ts_len)}
            idx_class += 1

    # create the dataset
    for i in range(ts_n):
        # for the train
        c = y_train[i]
        curr_pattern_pos = class_def[c]['pattern_pos']
        curr_pattern_len = class_def[c]['pattern_len']
        x_train[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] = \
            x_train[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] + 1.0

        # for the test
        c = y_test[i]
        curr_pattern_pos = class_def[c]['pattern_pos']
        curr_pattern_len = class_def[c]['pattern_len']
        x_test[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] = \
            x_test[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] + 1.0

    # znorm
    x_train = (x_train - x_train.mean(axis=1, keepdims=True)) \
              / x_train.std(axis=1, keepdims=True)

    x_test = (x_test - x_test.mean(axis=1, keepdims=True)) \
             / x_test.std(axis=1, keepdims=True)

    # visualize example
    # plt.figure()
    # colors = generate_array_of_colors(nb_classes)
    # for c in range(nb_classes):
    #     plt.plot(x_train[y_train == c][0], color=colors[c], label='class-' + str(c))
    # plt.legend(loc='best')
    # plt.savefig('out.pdf')
    # exit()

    # np.save(out_dir+'x_train.npy',x_train)
    # np.save(out_dir+'y_train.npy',y_train)
    # np.save(out_dir+'x_test.npy',x_test)
    # np.save(out_dir+'y_test.npy',y_test)

    # print('Done creating dataset!')

    return x_train, y_train, x_test, y_test

In [ ]:
def generate_array_of_colors(n):
    # https://www.quora.com/How-do-I-generate-n-visually-distinct-RGB-colours-in-Python
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    alpha = 1.0
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append((r / 255, g / 255, b / 255, alpha))
    return ret

In [ ]:
# def read_sits_xps(root_dir):
#     datasets_dict = {}
#     path_to_data = root_dir + 'archives/SITS/resampled-SITS/'
#     path_to_test = root_dir + 'archives/SITS/' + 'SatelliteFull_TEST_1000.csv'

#     x_test, y_test = readsits(path_to_test)

#     for subdir, dirs, files in os.walk(path_to_data):
#         for file_name in files:
#             arr = file_name.split('.')
#             dataset_name = arr[0]
#             file_type = arr[1]
#             if file_type == 'csv':
#                 x_train, y_train = readsits(subdir + '/' + file_name)

#                 datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
#                                                y_test.copy())

#     return datasets_dict

In [ ]:
def resample_dataset(x, rate):
    new_x = np.zeros(shape=(x.shape[0], rate))
    from scipy import signal
    for i in range(x.shape[0]):
        f = signal.resample(x[0], rate)
        new_x[i] = f
    return new_x

In [ ]:
def run_length_xps(root_dir):
    #archive_name = ARCHIVE_NAMES[0]
    archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
    dataset_name = 'InlineSkate'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)

    lengths = [2 ** i for i in range(5, 12)]

    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]
    x_val = datasets_dict[dataset_name][4]
    y_val = datasets_dict[dataset_name][5]

    new_archive_name = 'TSC'
#    new_archive_name = 'InlineSkateXPs'

    for l in lengths:
        new_x_train = resample_dataset(x_train, l)
        new_x_test = resample_dataset(x_test, l)
        new_x_val = resample_dataset(x_val, l)
        new_dataset_name = dataset_name + '-' + str(l)
        new_dataset_dir = root_dir + 'archives/' + new_archive_name + '/' + new_dataset_name + '/'
        create_directory(new_dataset_dir)

        np.save(new_dataset_dir + 'x_train.npy', new_x_train)
        np.save(new_dataset_dir + 'y_train.npy', y_train)
        np.save(new_dataset_dir + 'x_test.npy', new_x_test)
        np.save(new_dataset_dir + 'y_test.npy', y_test)
        np.save(new_dataset_dir + 'x_val.npy', new_x_val)
        np.save(new_dataset_dir + 'y_val.npy', y_val)

In [ ]:
UNIVARIATE_DATASET_NAMES = ['AllCandidas']

UNIVARIATE_ARCHIVE_NAMES = ['TSC', 'InlineSkateXPs', 'SITS']

SITS_DATASETS = ['SatelliteFull_TRAIN_c301', 'SatelliteFull_TRAIN_c200', 'SatelliteFull_TRAIN_c451',
                 'SatelliteFull_TRAIN_c89', 'SatelliteFull_TRAIN_c677', 'SatelliteFull_TRAIN_c59',
                 'SatelliteFull_TRAIN_c133']

InlineSkateXPs_DATASETS = ['InlineSkate-32', 'InlineSkate-64', 'InlineSkate-128',
                           'InlineSkate-256', 'InlineSkate-512', 'InlineSkate-1024',
                           'InlineSkate-2048']

dataset_names_for_archive = {'TSC': UNIVARIATE_DATASET_NAMES,
                             'SITS': SITS_DATASETS,
                             'InlineSkateXPs': InlineSkateXPs_DATASETS}

In [ ]:
class Classifier_NNE:

    def create_classifier(self, model_name, input_shape, nb_classes, output_directory, verbose=False,
                          build=True):
        if self.check_if_match('inception*', model_name):
            #from classifiers import inception
            return Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose,build=build)

    def check_if_match(self, rex, name2):
        import re
        pattern = re.compile(rex)
        return pattern.match(name2)

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, nb_iterations=5,
                 clf_name='inception'):
        self.classifiers = [clf_name]
        out_add = ''
        for cc in self.classifiers:
            out_add = out_add + cc + '-'
        self.archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
        self.iterations_to_take = [i for i in range(nb_iterations)]
        for cc in self.iterations_to_take:
            out_add = out_add + str(cc) + '-'
        self.output_directory = output_directory.replace('nne',
                                                         'nne' + '/' + out_add)
        create_directory(self.output_directory)
        self.dataset_name = output_directory.split('/')[-2]
        self.verbose = verbose
        self.models_dir = output_directory.replace('nne', 'classifier')

    def fit(self, x_train, y_train, x_test, y_test, y_true):
        # no training since models are pre-trained
        start_time = time.time()

        y_pred = np.zeros(shape=y_test.shape)

        ll = 0

        # loop through all classifiers
        for model_name in self.classifiers:
            # loop through different initialization of classifiers
            for itr in self.iterations_to_take:
                if itr == 0:
                    itr_str = ''
                else:
                    itr_str = '_itr_' + str(itr)

                curr_archive_name = self.archive_name + itr_str

                curr_dir = self.models_dir.replace('classifier', model_name).replace(
                    self.archive_name, curr_archive_name)

                model = self.create_classifier(model_name, None, None,
                                               curr_dir, build=False)

                predictions_file_name = curr_dir + 'y_pred.npy'
                # check if predictions already made
                if check_if_file_exits(predictions_file_name):
                    # then load only the predictions from the file
                    curr_y_pred = np.load(predictions_file_name)
                else:
                    # then compute the predictions
                    curr_y_pred = model.predict(x_test, y_true, x_train, y_train, y_test,
                                                return_df_metrics=False)
                    keras.backend.clear_session()

                    np.save(predictions_file_name, curr_y_pred)

                y_pred = y_pred + curr_y_pred

                ll += 1

        # average predictions
        y_pred = y_pred / ll

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        duration = time.time() - start_time

        df_metrics = calculate_metrics(y_test, y_pred, duration)

        df_metrics.to_csv(self.output_directory + 'df_metrics_val.csv', index=False)

        gc.collect()

In [ ]:
class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(),
                      metrics= [
                                tf.keras.metrics.CategoricalAccuracy(name='accuracy', dtype=None),
                                tf.keras.metrics.Recall(),
                                tf.keras.metrics.Precision(),
                                tf.keras.metrics.TruePositives(),
                                tf.keras.metrics.TrueNegatives(),
                                tf.keras.metrics.FalsePositives(),
                                tf.keras.metrics.FalseNegatives()
                                ])
 
        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, plot_test_acc=False):
        #if len(keras.backend.tensorflow_backend._get_available_gpus()) == 0:       
        #    print('error no gpu')
        #    exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        print("Dentro do FitModel")
        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        if plot_test_acc:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)
        else:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
                              return_df_metrics=False)

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)
         # save predictions
        np.save(self.output_directory + 'y_true.npy', y_true)

        # convert the predicted from binary to integer
        print(f"y_pred before save =  \n {y_pred}")
        y_pred = np.argmax(y_pred, axis=1)

        df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration,
                               plot_test_acc=plot_test_acc)

        keras.backend.clear_session()

        return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_test, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

In [ ]:
def prepare_data():
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]
    x_val = datasets_dict[dataset_name][4]
    y_val = datasets_dict[dataset_name][5]        
    print(f"valor de x_train no prepare data {x_train}")
    print(f"valor de y_train no prepare data {y_train}")
    print(f"valor de y_val no prepare data {y_val}")
    nb_classes = len(np.unique(np.concatenate((y_train, y_test,y_val ), axis=0)))

    # make the min to zero of labels
    y_train, y_test, y_val = transform_labels(y_train, y_test, y_val)

    # save orignal y because later we will use binary
    y_true = y_test.astype(np.int64)
    y_true_train = y_train.astype(np.int64)
    y_true_val = y_val.astype(np.int64)
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(np.concatenate((y_train, y_test,y_val), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
    y_val = enc.transform(y_val.reshape(-1, 1)).toarray()

    if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
        x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    return x_train, y_train, x_test, y_test, x_test, x_val, y_val, y_true, nb_classes, y_true_train, enc

In [ ]:
def fit_classifier():
    input_shape = x_train.shape[1:]

    classifier = create_classifier(classifier_name, input_shape, nb_classes,
                                   output_directory)

    classifier.fit(x_train, y_train, x_test, y_test, y_true)

In [ ]:
def create_classifier(classifier_name, input_shape, nb_classes, output_directory,
                      verbose=False, build=True):
    if classifier_name == 'nne':
        #from classifiers import nne
        return Classifier_NNE(output_directory, input_shape, nb_classes, verbose)

    if classifier_name == 'inception':
        #from classifiers import inception
        return Classifier_INCEPTION(output_directory, input_shape, nb_classes,
                                    verbose, build=build)

In [ ]:
def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr

In [ ]:
############################################### main

#root_dir = '/b/home/uha/hfawaz-datas/temp-dl-tsc/'

root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas'

xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']

In [ ]:
import tensorflow as tf

In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

In [ ]:
# run nb_iter_ iterations of Inception on the whole TSC archive
classifier_name = 'inception'
#archive_name = ARCHIVE_NAMES[0]
archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
nb_iter_ = 10

datasets_dict = read_all_datasets(root_dir, archive_name)

for iter in range(nb_iter_):
    print('\t\titer', iter)

    trr = ''
    if iter != 0:
        trr = '_itr_' + str(iter)

    tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + trr + '/'

    for dataset_name in dataset_names_for_archive[archive_name]:
        print('\t\t\tdataset_name: ', dataset_name)

        x_train, y_train, x_test, y_test, x_test, x_val, y_val, y_true, nb_classes, y_true_train, enc = prepare_data()
        # print(f"Valor de y_train: {y_train}")
        # print(f"Valor de x_train: {x_train}")


        output_directory = tmp_output_directory + dataset_name + '/'

        temp_output_directory = create_directory(output_directory)

        if temp_output_directory is None:
            print('Already_done', tmp_output_directory, dataset_name)
            continue

        try:
            with tf.device('/device:GPU:0'):
                fit_classifier()
                print('\t\t\t\tDONE')
        except:
            fit_classifier()
            print('\t\t\t\tDONE')

            # the creation of this directory means
            create_directory(output_directory + '/DONE')

#res['std'] = std
# run the ensembling of these iterations of Inception
# classifier_name = 'nne'

# datasets_dict = read_all_datasets(root_dir, archive_name)

# tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '/'

# for dataset_name in dataset_names_for_archive[archive_name]:
#     print('\t\t\tdataset_name: ', dataset_name)

#     x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()

#     output_directory = tmp_output_directory + dataset_name + '/'
#     with tf.device('/device:GPU:0'):
#         fit_classifier()

#     print('\t\t\t\tDONE')

std = np.std(accuracy)
root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas'

clf_name = 'inception'

path = '/'

def update_metrics(root_dir,path ,archive_name, dataset_name, std):
    output_dir = root_dir + path + archive_name + '/' \
                            + dataset_name + '/' + 'df_metrics.csv'
    #print(output_dir)
    #print('j = ',j,'i = ',i)
    if os.path.exists(output_dir):
        df_metrics = pd.read_csv(output_dir)
        df_metrics['std'] = std
        df_metrics.to_csv(output_dir + 'df_metrics_val.csv', index=False)
    return df_metrics

/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas
/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas_TRAIN.csv
/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas_TEST.csv
/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas_VAL.csv
Valor do Xtrain no read_all_datasets [[1.993e+04 5.877e+04 1.544e+04 ... 1.188e+03 4.239e+03 4.000e+00]
 [8.122e+04 7.305e+03 6.886e+03 ... 1.055e+04 7.302e+03 4.000e+00]
 [8.223e+04 7.305e+03 6.032e+03 ... 1.834e+04 7.841e+03 6.000e+00]
 ...
 [2.606e+04 4.686e+04 2.178e+04 ... 2.586e+03 5.592e+03 4.000e+00]
 [3.245e+04 4.572e+04 2.271e+04 ... 2.613e+03 5.596e+03 5.000e+00]
 [6.787e+04 1.842e+04 2.108e+04 ... 1.091e+04 6.971e+03 2.000e+00]]
Valor do y_train no read_all_datasets [1. 6. 1. 2. 5. 1. 2. 2. 1. 2. 4. 5. 3. 1. 3. 5. 2. 5. 1. 6. 2. 6. 1. 5.
 1. 2. 2. 6. 6. 5. 2. 1. 5. 4. 1. 1. 1. 6. 1. 1. 5. 6. 1. 2. 6. 4. 2. 2.
 1. 

<ipython-input-15-c033a5082e59>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],


Dentro do FitModel
2/2 [==============================] - 1s 30ms/step
y_pred before save =  
 [[8.21470735e-07 9.99926925e-01 3.19230116e-06 4.85431083e-05
  4.00499312e-06 1.64947887e-05]
 [3.08247513e-06 9.99696493e-01 1.04843084e-05 2.43640709e-04
  1.10821511e-05 3.53343676e-05]
 [2.21175682e-07 1.41997261e-05 9.99969363e-01 9.83273185e-06
  4.48564288e-06 1.90254593e-06]
 [1.31589450e-05 2.75345137e-05 6.35775214e-05 9.99887705e-01
  6.21140680e-06 1.92610810e-06]
 [5.53234759e-06 6.58837525e-05 3.01612799e-05 2.50658081e-06
  2.17033339e-05 9.99874234e-01]
 [1.28879110e-05 2.54523984e-05 5.74117678e-07 1.10376652e-06
  6.83113904e-05 9.99891639e-01]
 [1.31168053e-06 9.99913692e-01 2.18213199e-06 6.62502207e-05
  1.18280586e-05 4.73000728e-06]
 [5.54030385e-06 8.18030749e-05 4.37379131e-05 2.69464908e-06
  2.16676381e-05 9.99844551e-01]
 [9.99825060e-01 1.86255293e-05 1.28438643e-07 1.29565640e-04
  2.07224530e-05 5.93202867e-06]
 [1.52154414e-07 9.99977112e-01 2.93121150e-09 3.4

<ipython-input-15-c033a5082e59>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],


Dentro do FitModel
2/2 [==============================] - 1s 31ms/step
y_pred before save =  
 [[1.66298719e-06 9.99962926e-01 1.22174679e-05 2.01963248e-05
  1.03512184e-06 1.97785425e-06]
 [7.19210129e-06 9.99735057e-01 5.30862089e-05 1.92015839e-04
  5.36221205e-06 7.32440458e-06]
 [2.30934205e-07 1.64063749e-05 9.99952078e-01 1.65533074e-05
  1.43373654e-05 2.97985650e-07]
 [4.65400859e-07 4.90181901e-06 2.53971234e-06 9.99978423e-01
  1.35540804e-05 1.09937673e-07]
 [6.89614990e-06 9.52934715e-06 5.97003091e-05 2.98133932e-06
  4.00436002e-05 9.99880791e-01]
 [1.10036390e-05 1.98840899e-05 8.32896603e-07 1.03186972e-06
  4.01848192e-05 9.99927044e-01]
 [1.72791169e-06 9.99964356e-01 6.17962769e-06 1.98258676e-05
  7.71216855e-06 2.95127734e-07]
 [8.99068436e-06 1.40596730e-05 1.11154099e-04 4.21939058e-06
  4.25642538e-05 9.99819100e-01]
 [9.99598086e-01 3.01571912e-04 2.83915347e-06 3.37769279e-06
  5.36738917e-05 4.04747771e-05]
 [5.02530838e-07 9.99962687e-01 6.06239894e-08 8.6

...

<ipython-input-15-c033a5082e59>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],


y_pred before save =  
 [[6.20293167e-06 9.99947071e-01 4.27609984e-06 2.12385494e-05
  7.63181015e-06 1.36348408e-05]
 [1.51685454e-05 9.99830365e-01 1.59031715e-05 9.42296028e-05
  2.03373620e-05 2.39562305e-05]
 [1.42897989e-07 2.36130491e-05 9.99881983e-01 6.69390211e-05
  2.68059011e-05 4.84763859e-07]
 [3.89662944e-07 6.40114886e-05 1.40712646e-05 9.99915600e-01
  5.97740200e-06 2.46552325e-08]
 [2.90662410e-05 3.16798178e-05 4.00799399e-05 7.80415576e-07
  3.08368035e-05 9.99867558e-01]
 [1.89983421e-05 2.96841481e-05 6.01847432e-06 1.91710728e-06
  7.05319631e-04 9.99238133e-01]
 [2.95738459e-06 9.99949217e-01 5.13729674e-07 1.84370710e-05
  2.56761887e-05 3.14138060e-06]
 [3.28321512e-05 4.72169741e-05 8.01341594e-05 1.26592079e-06
  3.76090575e-05 9.99800861e-01]
 [9.99979377e-01 1.47147603e-06 4.95750534e-08 8.12656751e-08
  4.14342730e-06 1.49431544e-05]
 [2.65257731e-05 9.99727070e-01 3.47833193e-08 4.79992468e-06
  6.88640575e-05 1.72592088e-04]
 [4.78489540e-07 2.3850905

2/2 [==============================] - 1s 26ms/step
y_pred before save =  
 [[1.90561298e-06 9.99937892e-01 1.14504273e-05 3.83349143e-05
  1.30346587e-06 9.01408475e-06]
 [4.72069905e-06 9.99815047e-01 3.39242360e-05 1.27615247e-04
  3.62863398e-06 1.51179975e-05]
 [4.05410425e-08 3.83580773e-05 9.99907851e-01 3.17132908e-05
  2.06688073e-05 1.37592906e-06]
 [3.62893161e-06 1.23307211e-04 9.55328214e-05 9.99683380e-01
  9.29496091e-05 1.24245753e-06]
 [1.56073453e-04 9.01833409e-05 6.68245484e-05 1.57629638e-05
  2.62332378e-05 9.99644876e-01]
 [3.38432845e-04 9.89714754e-05 8.94291770e-06 2.54060997e-05
  2.41016722e-04 9.99287307e-01]
 [1.48153117e-06 9.99892592e-01 1.53112433e-05 8.16032407e-05
  6.87638567e-06 2.15745172e-06]
 [1.80876697e-04 1.20474579e-04 1.16146788e-04 2.17194811e-05
  2.81521898e-05 9.99532700e-01]
 [9.99861956e-01 5.14907661e-05 1.09558513e-08 2.11986588e-07
  7.53469576e-05 1.09888924e-05]
 [2.59358046e-07 9.99998927e-01 1.92486493e-09 4.08559551e-08
  4.520

<ipython-input-15-c033a5082e59>:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],


Dentro do FitModel


In [ ]:

def gen_results_csv(clf_name, path, root_dir):

    ARCHIVE_NAMES = ['TSC', 'TSC_itr_1', 'TSC_itr_2', 'TSC_itr_3', 'TSC_itr_4', 'TSC_itr_5',
                     'TSC_itr_6', 'TSC_itr_7', 'TSC_itr_8', 'TSC_itr_9'] 
    
    DATASET_NAME = ['AllCandidas'] 
   
    res1 = np.zeros((10, 3))
    
    res1 = pd.DataFrame(res1)
    
    res1 = res1.astype(object)
    
    nomes = ['classifier_name', 'archive_name', 'dataset_name']
    res1.columns=nomes
    
    res2 = np.zeros((10, 9))
    
    i = 0
    j = 0
    for archive_name in ARCHIVE_NAMES:
    
        o = 0
        for dataset_name in DATASET_NAME:
    
            output_dir = root_dir + path + archive_name + '/' \
                         + dataset_name + '/' + 'df_metrics_val.csv'
            #print(output_dir)
            print('j = ',j,'i = ',i)            
            if not os.path.exists(output_dir):
                print(f"Path {output_dir} do not exists")
                continue
            df_metrics = pd.read_csv(output_dir)
            print(f"Hehehe {df_metrics}")
            res1['classifier_name'][j] = clf_name
            res1['archive_name'][j] = archive_name
            res1['dataset_name'][j] = dataset_name
            res2[j,0] = j
            res2[j,1] = o
            res2[j,2] = i
            res2[j,3] = df_metrics['precision']
            res2[j,4] = df_metrics['accuracy']
            res2[j,5] = df_metrics['recall']
            res2[j,6] = df_metrics['f1_score']
            res2[j,7] = np.std(df_metrics['accuracy'])
            res2[j,8] = df_metrics['duration']
            
            o += 1
            j += 1
        i += 1
    
    
    res2 = pd.DataFrame(res2)
    #print(res2)
    nomes = ['ord1', 'ord2', 'iteration', 'precision', 'accuracy','recall','f1_score','std','duration']
    res2.columns=nomes
    
    
    res = pd.concat([res1, res2], ignore_index=True, axis=1)
    nomes = ['classifier_name', 'archive_name', 'dataset_name','ord1', 'ord2',
             'iteration', 'precision', 'accuracy', 'recall','f1_score','std', 'duration']
    res.columns=nomes
    
    res = res.sort_values(by=['ord2', 'ord1'])
    
    return res


root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas/results/inception'

clf_name = 'inception'

path = '/'

res = gen_results_csv(clf_name, path, root_dir)

print(res)

arq = clf_name + '.csv'
filepath = root_dir + path + arq
res.to_csv(filepath, index=False)


In [ ]:
from pickle import dump
def save_results(df_to_save, name, train_val_test):
    print("Saving metrics...")
    print(f"Df accuracy: {df_to_save}")
    df_to_save.to_csv(root_dir+ '/df_'+name+'_'+train_val_test+'2.csv', index=False)
    print("Accuracy of models:")
    print(df_to_save)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import time
def test_final_model(x_val, y_val, model):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
#     for name, model in models:
        # df_test = cross_validate(model, X_test, y_test, cv=kf, scoring=scoring
        #                          , return_train_score=True)
#         print(f'Model in test: {name}')

    df_test = pd.DataFrame([], columns=['accuracy','recall','precision','f1_score','specificity','testTime'])
    
    start_time = time.time()
    y_pred = model.predict(x_val, batch_size=1500)
    time_prediction = time.time()-start_time

    y_pred = np.argmax(y_pred, axis=1)
    onehot_encoder = OneHotEncoder(sparse=False)
    y_pred = y_pred.reshape(len(y_pred), 1)
    y_pred = onehot_encoder.fit_transform(y_pred)
    print("Y pred ---- ")
    print(y_pred)
    print("Y val")
    print(y_val)
    accuracy  =  accuracy_score(y_val, y_pred)
    recall    =  recall_score(y_val, y_pred, average='macro')
    precision =  precision_score(y_val, y_pred, average='macro')
    f1  =  f1_score(y_val, y_pred, average='macro')
    specificity = specificity_score(y_val, y_pred)
    print(f"accuracy: {accuracy}, recall: {recall}, precision: {precision},f1: {f1} ")
    df_test.at[0, 'accuracy'] = accuracy
    df_test.at[0, 'recall'] = recall
    df_test.at[0,'precision'] = precision
    df_test.at[0,'f1_score'] = f1
    df_test.at[0,'specificity'] = specificity
    df_test.at[0,'testTime'] =  time_prediction
    print(df_test.keys())
    save_results(df_test,"INCEPTION", 'test')
    print("Finished test!")

In [ ]:
archive_name = 'TSC_itr_3'
tmp_output_directory = root_dir + '/'+ archive_name + '/'
output_directory = tmp_output_directory + dataset_name + '/'
print(tmp_output_directory)
# dataset = np.load(output_directory+'/y_pred'+'.npy', encoding='bytes')
# dataset

In [ ]:
print(x_val.shape)
print(y_val.shape)

In [ ]:
model_path = output_directory + 'best_model.hdf5'
model = keras.models.load_model(model_path)
test_final_model(x_val, y_val, model)